In [5]:
pip install dash-bootstrap-components

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install jupyter_dash

Note: you may need to restart the kernel to use updated packages.


In [121]:
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dash_table.Format import Group
from dash_table import DataTable

In [122]:
data = pd.read_csv('131000961.csv')

healthcan_df = data.pivot( index= ['GEO', 'Age group', 'Sex', 'Characteristics', 'REF_DATE', 'UOM'],
                          columns= 'Indicators', values='VALUE').reset_index()

table_df = data.pivot( index= ['GEO', 'Age group', 'Sex', 'Indicators', 'REF_DATE', 'UOM'],
                          columns= 'Characteristics', values='VALUE').reset_index()

In [123]:
#years list
years = ['2015', '2016', '2017', '2018', '2019', '2020']

#indicator list
data_pivot = data.pivot( index= ['GEO', 'Age group', 'Sex', 'Characteristics', 'REF_DATE', 'UOM'], columns= 'Indicators', values='VALUE').reset_index()
data_pivot['REF_DATE'] = data_pivot['REF_DATE'].astype(int)
indicatorList =  data_pivot.columns.tolist() #this works
indicatorList.remove('GEO')
indicatorList.remove('Age group')
indicatorList.remove('Sex')
indicatorList.remove('Characteristics')
indicatorList.remove('REF_DATE')
indicatorList.remove('UOM')

#age group list
data_pivot = data.pivot( index= ['GEO', 'Indicators', 'Sex', 'Characteristics', 'REF_DATE', 'UOM'], columns= 'Age group', values='VALUE').reset_index()
data_pivot['REF_DATE'] = data_pivot['REF_DATE'].astype(int)
agegroupList =  data_pivot.columns.tolist()
agegroupList.remove('GEO')
agegroupList.remove('Indicators')
agegroupList.remove('Sex')
agegroupList.remove('Characteristics')
agegroupList.remove('REF_DATE')
agegroupList.remove('UOM')

agegroupList.insert(0, agegroupList.pop())
ageDict = {i : agegroupList[i] for i in range(0, len(agegroupList))}

#sex list
data_pivot = data.pivot( index= ['GEO', 'Indicators', 'Age group', 'Characteristics', 'REF_DATE', 'UOM'], columns= 'Sex', values='VALUE').reset_index()
data_pivot['REF_DATE'] = data_pivot['REF_DATE'].astype(int)
sexList =  data_pivot.columns.tolist()
sexList.remove('GEO')
sexList.remove('Indicators')
sexList.remove('Age group')
sexList.remove('Characteristics')
sexList.remove('REF_DATE')
sexList.remove('UOM')

sexList.insert(1, sexList.pop())
sexDict = {i : sexList[i] for i in range(0, len(sexList))}

#character list
data_pivot = data.pivot( index= ['GEO', 'Indicators', 'Age group', 'Sex', 'REF_DATE', 'UOM'], columns= 'Characteristics', values='VALUE').reset_index()
data_pivot['REF_DATE'] = data_pivot['REF_DATE'].astype(int)
characteristicList =  data_pivot.columns.tolist()
characteristicList.remove('GEO')
characteristicList.remove('Indicators')
characteristicList.remove('Age group')
characteristicList.remove('Sex')
characteristicList.remove('REF_DATE')
characteristicList.remove('UOM')

#UOM list
data_pivot = data.pivot( index= ['GEO', 'Indicators', 'Sex', 'Characteristics', 'REF_DATE', 'Age group'], columns= 'UOM', values='VALUE').reset_index()
data_pivot['REF_DATE'] = data_pivot['REF_DATE'].astype(int)
uomlist =  data_pivot.columns.tolist()
uomlist.remove('GEO')
uomlist.remove('Indicators')
uomlist.remove('Sex')
uomlist.remove('Characteristics')
uomlist.remove('REF_DATE')
uomlist.remove('Age group')

uomDict = {i : uomlist[i] for i in range(0, len(uomlist))}

grouplist = ['sex', 'age']
groupDict = {i : grouplist[i] for i in range(0, len(grouplist))}

In [124]:
import plotly.graph_objects as go
def make_empty_fig():
    fig = go.Figure()
    fig.layout.paper_bgcolor = '#EC8D8D'
    fig.layout.plot_bgcolor = '#EC8D8D'
    return fig

In [125]:
app = JupyterDash(__name__,
                  external_stylesheets=[dbc.themes.COSMO] ) 

app.layout = html.Div([
    html.H2('Canada Health dataset',style={'textAlign': 'center', 'fontWeight': 'bold', 'fontSize': '35px'}), #title at the top
    html.Br(), #break
    
    #row 1 that icludes the title indicator and the dropdown bar with indicators
    dbc.Row([
        dbc.Col([dbc.Label("Indicator:", style={'textAlign': 'left', 'fontWeight': 'bold', 'fontSize': '25px'}),
                 dcc.Dropdown(id='id_indicator', value="Body mass index, adjusted self-reported, adult (18 years and over), obese",
                              options=[{'label': ind, 'value': ind} for ind in indicatorList])], md=12, lg=12),
            ]),
    
    html.Br(), #break
    
    #row two that includes three sliders
    dbc.Row([
        dbc.Col([dbc.Label("Age group:"),
            dcc.Slider(id='id_age', min=0, max=5, step=None, 
                       dots=True, value =0, marks={0: {'label':'Total, 12 years and over','style':{'color': '#000000'}},
                                                   1: {'label':'12 to 17 years','style':{'color': '#000000'}},
                                                   2: {'label':'18 to 34 years','style':{'color': '#000000'}},
                                                   3: {'label':'35 to 49 years','style':{'color': '#000000'}},
                                                   4: {'label':'50 to 64 years','style':{'color': '#000000'}},
                                                   5: {'label':'65 years and over','style':{'color': '#000000'}}}, included=False)], md=5, lg=5),
        dbc.Col([dbc.Label("Sex group:"),
            dcc.Slider(id='id_sex', min=0, max=2, step=None,
                       dots=True, value =0, marks={0: {'label':'Both sexes','style':{'color': '#000000'}},
                                                   1: {'label':'Males','style':{'color': '#000000'}},
                                                   2: {'label':'Females','style':{'color': '#000000'}}}, included=False)], md=3, lg=3),
        dbc.Col([dbc.Label("Units:"),
            dcc.Slider(id='id_units', min=0, max=1, step=None,
                       dots=True, value =0, marks={0: {'label':'Number','style':{'color': '#000000'}},
                                                   1: {'label':'Percent','style':{'color': '#000000'}}}, included=False)], md=2, lg=2),
            ]),
    
    html.Br(),
    
    #row three (no break before) - has year bar in it
    html.H2('Section 1',style={'textAlign': 'left', 'fontWeight': 'bold', 'fontSize': '25px'}),
    dbc.Row([
        dbc.Col([dbc.Label("Year:"),
    dcc.Dropdown(id='id_year',
                 value="2015",
                 options= [{'label': year,'value': year} for year in healthcan_df['REF_DATE'].unique()])
    ]),
        ]),
    
    #this row has two figures and a table
    dbc.Row([
        dbc.Col([dcc.Graph(id='id_chart_one')], lg=4),
        dbc.Col([dcc.Graph(id='id_chart_two')], lg=4),
        dbc.Col([html.Div( id='id_table')], lg=4),
        ]),
    
    html.H2('Section 2',style={'textAlign': 'left', 'fontWeight': 'bold', 'fontSize': '25px'}),
    dbc.Row([
        dbc.Col([dbc.Label('Years:'),
                 dcc.Dropdown(id='id_yearsdrop', 
                              multi=True,
                              placeholder='Select one or more years',
                              options=[{'label': yr,'value': yr} for yr in healthcan_df['REF_DATE'].unique()]), ], md=12, lg=7),
        dbc.Col([dbc.Label("Group:"),
                 dcc.Slider(id='id_agesex', min=0, max=1, step=1,
                            dots=True, value =0, marks={0: {'label':'sex','style':{'color': '#000000'}}, 1: {'label':'age','style':{'color': '#000000'}}}, included=False)], md=3, lg=3),
                ]),
    dbc.Row([
        dbc.Col([dcc.Graph(id='id_chart_three', figure=make_empty_fig())], lg=12)
                ]),
    
                    ], style={'backgroundColor': '#EC8D8D'})


@app.callback(Output(component_id='id_chart_one',component_property='figure'),
              Input(component_id='id_indicator', component_property='value'),
              Input(component_id='id_age', component_property='value'),
              Input(component_id='id_sex', component_property='value'),
              Input(component_id='id_units', component_property='value'),
              Input(component_id='id_year', component_property='value'))

def display_figure_one(indicator, age, sex, units, year):
    
    age1 = ageDict[age]
    sex1 = sexDict[sex]
    units1 = uomDict[units]
    
    df = (healthcan_df[(healthcan_df['Age group']==age1)&
                    (healthcan_df['Sex']==sex1)&
                    (healthcan_df['UOM']==units1)&
                    (healthcan_df['REF_DATE']==year)])
    
    n_prov = len(df['GEO'].unique())
    
    if units1 == 'Number':
        
        fig = px.bar(df[df['Characteristics']=="Number of persons"].sort_values(indicator),
                     x=indicator,
                     labels={"GEO":"Province", indicator:"Number of persons"},
                     y='GEO',
                     height=200 + (20*n_prov),
                     title=' - '.join([indicator, str(year)]),
                     orientation='h',
                     log_x=True)
        fig['layout']['title']['font'] = dict(size=13, color='black')
        fig.update_xaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig.update_yaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig.layout.paper_bgcolor = '#EC8D8D'
    
    else:
        
        fig = px.bar(df[df['Characteristics']=="Percent"].sort_values(indicator),
                     x=indicator,
                     labels={"GEO":"Province", indicator:"Percent"},
                     y='GEO',
                     height=200 + (20*n_prov),
                     title=' - '.join([indicator, str(year)]),
                     orientation='h',
                     log_x=True)
        fig['layout']['title']['font'] = dict(size=13, color='black')
        fig.update_xaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig.update_yaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig.update_xaxes(ticksuffix="%")
        fig.layout.paper_bgcolor = '#EC8D8D'
    
    return fig

@app.callback(Output(component_id='id_chart_two',component_property='figure'),
              Input(component_id='id_indicator', component_property='value'),
              Input(component_id='id_age', component_property='value'),
              Input(component_id='id_sex', component_property='value'),
              Input(component_id='id_units', component_property='value'),
              Input(component_id='id_year', component_property='value') )

def display_figure_two(indicator, age, sex, units, year):
    
    age1 = ageDict[age]
    sex1 = sexDict[sex]
    units1 = uomDict[units]
    
    df = (healthcan_df[(healthcan_df['Age group']==age1)&
                        (healthcan_df['Sex']==sex1)&
                        (healthcan_df['UOM']==units1)&
                        (healthcan_df['REF_DATE']==year)])
    
    n_prov = len(df['GEO'].unique())
    
    if units1 == 'Number':
        
        list_of_values_num = ['High 95% confidence interval, number of persons',
                  'Low 95% confidence interval, number of persons',
                  'Number of persons']
        
        fig1 = px.scatter(df[df['Characteristics'].isin(list_of_values_num)].sort_values(indicator),
        x=indicator,
        y='GEO',
        labels={"GEO":"Province", indicator:""},
        height=200 + (20*n_prov),
        title=' - '.join([indicator, str(year)]),
        width=700,
        log_x=True,
        color='Characteristics',
        symbol='Characteristics')
        fig1.update_layout(legend = dict(font = dict(size = 8, color = "black")),
                           legend_title = dict(font = dict(size = 10, color = "black")))
        fig1['layout']['title']['font'] = dict(size=13, color='black')
        fig1.update_xaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig1.update_yaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig1.layout.paper_bgcolor = '#EC8D8D'
    
    else:
        
        list_of_values_perc = ['High 95% confidence interval, percent',
                          'Low 95% confidence interval, percent',
                          'Percent']
        
        fig1 = px.scatter(df[df['Characteristics'].isin(list_of_values_perc)].sort_values(indicator),
        x=indicator,
        y='GEO',
        labels={"GEO":"Province", indicator:""},
        height=200 + (20*n_prov),
        title=' - '.join([indicator, str(year)]),
        width=700,
        log_x=True,
        color='Characteristics',
        symbol='Characteristics')
        fig1.update_layout(legend = dict(font = dict(size = 8, color = "black")),
                           legend_title = dict(font = dict(size = 10, color = "black")))
        fig1['layout']['title']['font'] = dict(size=13, color='black')
        fig1.update_xaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig1.update_yaxes(tickfont_size=10, tickfont_color='black', titlefont_size=12, titlefont_color='black')
        fig1.update_xaxes(ticksuffix="%")
        fig1.layout.paper_bgcolor = '#EC8D8D'
    
    return fig1

@app.callback(Output(component_id='id_table',component_property='children'),
              Input(component_id='id_indicator', component_property='value'),
              Input(component_id='id_age', component_property='value'),
              Input(component_id='id_sex', component_property='value'),
              Input(component_id='id_year', component_property='value') )

def display_table(indicator, age, sex, year):
    
    sex1 = sexDict[sex]
    age1 = ageDict[age]
    
    res = table_df[table_df['REF_DATE']==year].filter(regex='REF_DATE|GEO|Sex|Age group|UOM|Indicator|Statistically')

    res = res.rename(columns={"Statistically different from previous reference period": "Sign. diff. from prev. year",
                              "Statistically different from the Canada (excluding territories) rate": "Sign. diff. from the federal",
                              "GEO": "Province",
                              "REF_DATE": "Year"})

    res = res.fillna('N/A')
    res['Sign. diff. from prev. year'] = res['Sign. diff. from prev. year'].replace({0: "No", -1: "N/A", 1: "Yes"})
    res['Sign. diff. from the federal'] = res['Sign. diff. from the federal'].replace({0: "No", -1: "N/A", 1: "Yes"})

    res = (res[(res['Indicators']==indicator)&
               (res['Sex']==sex1)&
               (res['Age group']==age1)&
               (res['UOM']=="Number")])

    res = res[['Year', 'Province', 'Sign. diff. from prev. year', 'Sign. diff. from the federal']]
    
    table = dbc.Col([DataTable(data=res.to_dict('records'),
                               columns=[{'name': col, 'id': col} for col in res.columns],
                               style_header={'whiteSpace': 'normal'}, #allow the header to overflow for long text
                               fixed_rows={'headers': True}, #When rolling, allows the headers to be in a fix position.
                               virtualization=False, #allow virtualization, app load a portion of the data (for long talbles)
                               style_as_list_view=True,
                               style_table={'height': '400px', 'backgroundColor': '#EC8D8D', 'margin-left': '45px'},
                               style_cell={'fontSize':12, 'fontColor':'black', 'backgroundColor': '#EC8D8D', 'height': '10px'})
                    ], lg =10)
    
    return table

@app.callback(Output(component_id='id_chart_three',component_property='figure'),
              Input(component_id='id_indicator', component_property='value'),
              Input(component_id='id_yearsdrop', component_property='value'),
              Input(component_id='id_agesex', component_property='value'))

def display_figure_three(indicator, years, agesex):
    
    agesex1 = groupDict[agesex]
    df = (healthcan_df[(healthcan_df['Characteristics']=='Percent')&(healthcan_df['REF_DATE'].isin(years))])

    if agesex1 == 'sex':
    
        fig2 = px.histogram(data_frame=df, x=indicator, color ='Sex', facet_col='Sex', facet_col_wrap=3, facet_row='REF_DATE', histnorm='percent', category_orders={"Sex": ['Females', 'Both sexes', 'Males']})
        fig2.update_xaxes(title='')
        fig2.update_yaxes(title_font_color="black", title_font_size=12)
        fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig2.add_annotation(x=0.5, y=-0.15, text=indicator, xref="paper", yref="paper", showarrow=False, font_color="black", font_size=12)
        fig2.for_each_xaxis(lambda x: x.update(title=''))
        fig2.update_xaxes(ticksuffix="%", showticksuffix="all", tickfont_size=10, color='black')
        fig2.layout.paper_bgcolor = '#EC8D8D'
        fig2.update_layout(legend = dict(font = dict(size = 10, color = "black")), legend_title = dict(font = dict(size = 12, color = "black")))
        fig2['layout']['yaxis']['title']['text']='Relative Frequency'

    else:
    
        fig2 = px.histogram(data_frame=df, x=indicator, color ='Age group', facet_col='Age group', facet_col_wrap=5, facet_row='REF_DATE', histnorm='percent', category_orders={"Age group": ['12 to 17 years','65 years and over', 'Total, 12 years and over', '50 to 64 years', '18 to 34 years', '35 to 49 years']})
        fig2.update_xaxes(title='')
        fig2.update_yaxes(title_font_color="black", title_font_size=12)
        fig2.add_annotation(x=0.5, y=-0.15, text=indicator, xref="paper", yref="paper", showarrow=False, font_color="black", font_size=12)
        fig2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig2.for_each_xaxis(lambda x: x.update(title=''))
        fig2.update_xaxes(ticksuffix="%", showticksuffix="all", tickfont_size=10, color='black')
        fig2.layout.paper_bgcolor = '#EC8D8D'
        fig2.update_layout(legend = dict(font = dict(size = 10, color = "black")), legend_title = dict(font = dict(size = 12, color = "black")))
        fig2['layout']['yaxis']['title']['text']='Relative Frequency'
        
        
    return fig2

if __name__ == '__main__':
    app.run_server(mode='external', debug=False)

C:\Users\marcj\anaconda3\envs\624w9\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



Dash app running on http://127.0.0.1:8050/
